# **1. Import Library**

In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **2. Inference**

In [ ]:
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = load_model('LSTM_model.h5')

stress_labels = {0: "Low", 1: "Medium", 2: "High"}
emotion_labels = {0: "Anxious", 1: "Lonely", 2: "Depressed", 3: "Overwhelmed", 4: "Panicked"}

def preprocess_text(text, tokenizer, maxlen=500):
    sequences = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
    return padded

def predict(text):
    input_seq = preprocess_text(text, tokenizer)
    prediction = model.predict(input_seq)

    if isinstance(prediction, np.ndarray) and prediction.shape[1] == 8:
        stress_pred = np.argmax(prediction[0][:3])
        emotion_pred = np.argmax(prediction[0][3:])
        return stress_labels[stress_pred], emotion_labels[emotion_pred]

    elif isinstance(prediction, list) and len(prediction) == 2:
        stress_probs, emotion_probs = prediction
        stress_pred = np.argmax(stress_probs[0])
        emotion_pred = np.argmax(emotion_probs[0])
        return stress_labels[stress_pred], emotion_labels[emotion_pred]

    else:
        raise ValueError(f"Unexpected prediction shape: {type(prediction)}, {prediction}")

In [ ]:
def calculate_stress_level(model, text_input, vectorized_input):
    prediction = model.predict(vectorized_input)

    if not isinstance(prediction, list) or len(prediction) != 2:
        raise ValueError("Model must return a list of two arrays: [stress_probs, emotion_probs]")

    stress_probs, _ = prediction
    stress_probs = stress_probs[0]

    stress_score = stress_probs[0] * 0 + stress_probs[1] * 50 + stress_probs[2] * 100

    low_words = {
        "calm", "okay", "fine", "tire", "bore", "good", "sleepy", "irritate",
        "down", "unmotivate", "lazy", "dull", "frustrate", "annoy", "slightly", "upset",
        "restless", "uneasy", "discontent", "displease"
    }

    med_words = {
        "worry", "anxious", "exhaust", "fatigue", "sadness", "disgust", "disappoint",
        "miserable", "numb", "scare", "terrify", "stress", "anxiety", "cry", "helpless",
        "lose", "motivation", "sleep", "overstress", "pressure", "trigger", "overwhelm",
        "tense", "fearful", "panic", "unsettle", "concern", "distress", "worried", "cant breathe"
    }

    high_words = {
        "worthless", "suicide", "die", "depress", "depression", "isolate", "panic",
        "breakdown", "suffer", "despair", "hopeless", "gaslight", "abuse", "self",
        "harm", "kill", "sick", "ugly", "insecure", "insecurity", "grief", "disorder",
        "assault", "guilt", "paranoia", "nightmare", "reject", "miserable",
        "traumatize", "ptsd", "psychotic", "homicidal", "suicidal", "delusional",
        "cripple", "break", "victimize", "devastate", "abandon"
    }

    def word_based_score(text):
        words = set(text.lower().split())
        low_count = len(words & low_words)
        med_count = len(words & med_words)
        high_count = len(words & high_words)
        total = low_count + med_count + high_count
        if total == 0:
            return 0
        return (high_count * 100 + med_count * 50 + low_count * 0) / total

    combined_score = 0.8 * stress_score + 0.2 * word_based_score(text_input)
    return round(combined_score, 2)


## Inference Test #1

In [ ]:
input_story = "long post im sorry mom relationship always issue highlight fly monkey demand dollars month marry literally like dollars max time call high andor drink forgive steal son leave alone could literally hear adult daughter background year marry offer sister free car grandparents give us old one still break call talk much neglect month literally talk day speak two weeks time try pressure interact tictok initially take answer even point overbearingness literally chase another app forgive set boundaries even though would get real mad every time say talk things calmly neglectful put hubs raise sister also take care understand complex feelings try get go therapy multiple time incidents badly impact relationship already rocky due stuff break point incident havent able get past pressure move full time actually offer move fulltime care desperately need mental physically issue stress never return school next year table vehemently refuse stress mean wont able years agree well guess get clinical psych phd program arent aware things like full time job people can not work program im breadwinner basically two full time job part time job squirrel away case quit fulltime job due burnout add two months program start date spinal surgery caretaker month program hubby still isnt allow drive mom say fell wont go doc drive hour half take er sunday stay hours leave sister come minutes home turn around get threaten suicide discharge get different er get commit geriatric inpatient unit finally get home get work next day im also autistic adjust change overwhelm point im beyond tap plan take week school start months burnout set go back work week inpatient unit call demand take get spend seven days run around deal house two days hubs big talk absolutely could stay long term us reason week start school move repeatedly pressure sob beg understand cant shes us months need fulltime care need put mediwhatever need find fulltime place take hell due inpatient stay due threats selfharm fun fact fullblown catatonic episode last weeks closest years experience hubs go survival mode time would luck talk brick wall go home hub like hell let loose middle mugshots insist talk wasnt snap back normal trip drop justify action never whisperyelled much life worst justification fact basically gamble lose momfamily meus decide gamble meus course prioritize abusive neglectful family wife emotionally financially physically support last eight years family ever nearly fourty years conversation break break us felt love leave use fullbody experience im barely get ankles wet still listen first time ever could tell close break finally agree therapy us start marriage therapy know hes kill work set boundaries mil family growth last months hes entire adulthood tell doesnt matter fundamentally break relationship hes everything right doesnt matter say sweetest things much love us lie say cause want true lot time fight back feelings hate actually come back want really therapy therapy marriage therapy arent cut moments love still deminished like cant make get past take upset complaints cause know screw months since break conversation happen theres barely improvement time lose close family member hes gear another spinal surgery much timeenergy give salvage therapist hopeful im sure"
input_seq = preprocess_text(input_story, tokenizer)

stress, emotion = predict(input_story)
print(f"Predicted Stress Level: {stress}")
print(f"Predicted Emotion Label: {emotion}")

stress_level = calculate_stress_level(model, input_story, input_seq)
print(f"Calculated Stress Level (0–100): {stress_level}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Predicted Stress Level: High
Predicted Emotion Label: Overwhelmed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Calculated Stress Level (0–100): 92.47000122070312


## Inference Test #2

In [ ]:
input_story = "I have many problems in my life, but I don’t have anyone to confide in. All these problems piling up inside me are making me so stressed that I feel like I want to end my life."
input_seq = preprocess_text(input_story, tokenizer)

stress, emotion = predict(input_story)
print(f"Predicted Stress Level: {stress}")
print(f"Predicted Emotion Label: {emotion}")

stress_level = calculate_stress_level(model, input_story, input_seq)
print(f"Calculated Stress Level (0–100): {stress_level}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Predicted Stress Level: Medium
Predicted Emotion Label: Lonely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Calculated Stress Level (0–100): 40.13999938964844


## Inference Test #3

In [ ]:
input_story = "I've broke up with my girlfriend, and i just need someone to talk right now"
input_seq = preprocess_text(input_story, tokenizer)

stress, emotion = predict(input_story)
print(f"Predicted Stress Level: {stress}")
print(f"Predicted Emotion Label: {emotion}")

stress_level = calculate_stress_level(model, input_story, input_seq)
print(f"Calculated Stress Level (0–100): {stress_level}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Predicted Stress Level: Low
Predicted Emotion Label: Lonely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
Calculated Stress Level (0–100): 2.4000000953674316


## Inference #4

In [ ]:
input_story = "I failed all of my exams this semester, and I've scared that my parrents will be mad at me becuase of this"
input_seq = preprocess_text(input_story, tokenizer)

stress, emotion = predict(input_story)
print(f"Predicted Stress Level: {stress}")
print(f"Predicted Emotion Label: {emotion}")

stress_level = calculate_stress_level(model, input_story, input_seq)
print(f"Calculated Stress Level (0–100): {stress_level}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Predicted Stress Level: Medium
Predicted Emotion Label: Overwhelmed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Calculated Stress Level (0–100): 39.970001220703125


In [ ]:
input_story = "Every time I open social media, I feel this strange sense of fear, like something bad is waiting for me, even though I don't know what it is. It's this constant anxiety, like I'm about to see something that will ruin my mood or trigger thoughts I wasn’t prepared to face. Sometimes it’s fear of judgment, sometimes it’s just a vague unease, but it’s always there—lingering the moment I scroll."
input_seq = preprocess_text(input_story, tokenizer)

stress, emotion = predict(input_story)
print(f"Predicted Stress Level: {stress}")
print(f"Predicted Emotion Label: {emotion}")

stress_level = calculate_stress_level(model, input_story, input_seq)
print(f"Calculated Stress Level (0–100): {stress_level}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
Predicted Stress Level: Medium
Predicted Emotion Label: Lonely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
Calculated Stress Level (0–100): 50.38999938964844
